# Demo for Environment

##### Install required packages

In [ ]:
!pip install python-socketio
!pip install python-socketio[client]

##### Run the following server code on a separate process

In [ ]:
import socketio
import eventlet
import threading

sio = socketio.Server(cors_allowed_origins="*")
app = socketio.WSGIApp(sio)

client_sids = []


@sio.event
def connect(sid, environ):
    print("Client connected:", sid)
    client_sids.append(sid)


@sio.event
def disconnect(sid):
    print("Client disconnected:", sid)
    client_sids.remove(sid)


@sio.event
def make(sid, data):
    print("Received make from {}: {}".format(sid, data))
    # Forward the message to all other connected clients (except the sender)
    for client_sid in client_sids:
        if client_sid != sid:
            sio.emit("make", data, room=client_sid)


def run_server():
    eventlet.wsgi.server(eventlet.listen(("", 5000)), app)


if __name__ == "__main__":
    thread = threading.Thread(target=run_server)
    thread.start()
    print("Server is running in background")

##### Run the following code to intialize the client communication with the environment

In [ ]:
import socketio

sio = socketio.Client()


@sio.event
def connect():
    print("Connected to server")


@sio.event
def disconnect():
    print("Disconnected from the server")


@sio.event
def message(data):
    print("Received message:", data)


sio.connect("http://localhost:5000")

##### Make a environment

In [ ]:
# environment='Demo', num_env=[1], num_agent=[1]
data = {"environment": "Demo", "num_env": [1]}
sio.emit("make", data)

##### Reset the environment

In [ ]:
import json

with open("data.json", "r") as f:
    graph = json.load(f)

# graph, env_index=[0]
data = {"env_index": [0], "graph": graph}
sio.emit("reset", data)

##### Create a graph observation

In [ ]:
# obs_type='graph', radius=[-1]
data = {"type": "graph", "radius": [-1]}
sio.emit("observation", data)

##### Set action for an agent

In [ ]:
import utils

# (agent_index=[0], task=[0,0,0], coordinate=[0,0,0]
data = {"agent_index": agent_index, "task": task, "coordinate": coordinate}
sio.emit("set_action", data)

##### Change rendering settings

In [ ]:
# camera_index=[0], image_synthesis='raytracing', image_width=[256], image_height=[256], fps=10, visualize=False
data = {
    "camera_index": camera_index,
    "image_synthesis": image_synthesis,
    "image_width": image_width,
    "image_height": image_height,
    "fps": fps,
    "visualize": visualize,
}
sio.emit("render", data)

##### Change system agent settings

In [ ]:
# crowd=True, traffic=True
data = {"crowd": crowd, "traffic": traffic}
sio.emit("system_agent", data)